In [2]:
import pyspark

In [3]:
sc = pyspark.SparkContext()

## RDD 생성하기
pyspark에서 RDD를 생성하는 방법은 두 가지이다.

In [5]:
data = sc.parallelize(
    [('Amber', 22), ('Alfred', 23), ('Skye',4), ('Albert', 12), 
     ('Amber', 9)])

위와 같이 collection에 대해 parallelize() 함수를 수행하거나, 외부 어딘가 또는 특정 위치에 저장된 파일을 읽을 수 있다.

In [20]:
data_from_file = sc.textFile("D://Pyspark_dataset/VS14MORT.txt.gz",4)

데이터 다운로드 : http://tomdrabas.com/data/VS14MORT.txt.gz 

sc.textFile(...,n) 의 마지막 매개변수 n은 데이터셋이 나눠진 파티션의 개수를 의미한다. 책의 저자에 따르면 한 클러스터에서는 2~4개의 파티션으로 데이터셋을 나누는 것이 가장 좋다고 한다.

스파크는 NTFS, FAT, Mac OS Extended 또는 HDFS, S3, 카산드라 등의 분산파일시스템과 같은 여러 개의 파일시스템으로부터 데이터를 읽을 수 있다.

여러 가지 데이터 포맷이 지원된다. 텍스트, 파케이, JSON, 하이브테이블 그리고 관계형 데이터베이스에서의 데이터는 JDBC 드라이버로 읽을 수 있다. 스파크는 압축된 데이터셋도 읽어서 작업할 수 있다.

데이터가 어떻게 읽히느냐에 따라서, 데이터를 지니고 있는 객체는 조금씩 다르게 표현된다. 데이터가 읽히는 파일은 MapPartitionsRDD로 표현된다. 컬렉션에 대해 .parallelize() 함수를 돌릴 때처럼 ParallelCollectionRDD를 사용하지 않는다.

## Schema

DataFrame과 달리 RDD는 schema-less 데이터 구조이다. 그러므로 RDD를 사용할 때 데이터셋을 병렬 처리하는 것은 전혀 문제가 되지 않는다.

In [9]:
data_heterogenous = sc.parallelize([('Ferrari', 'fast'), {'Porsche': 100000}, ['Spain','visited', 4504]]).collect()
data_heterogenous

[('Ferrari', 'fast'), {'Porsche': 100000}, ['Spain', 'visited', 4504]]

그래서 tuple, dict, list 와 같은 거의 모든 데이터 구조를 섞을 수 있고, 이는 스파크에서 전혀 문제가 되지 않는다. 데이터셋에 대해 .collect() 함수를 수행하면(즉, 데이터셋을 드라이버에 다시 가져오기 위해 액션을 수행하면) 파이썬에서 일반적으로 했던 방식대로 객체 내의 데이터에 접근할 수 있다.

In [11]:
data_heterogenous[1]['Porsche']

100000

.collect() 함수는 RDD의 모든 엘리먼트를 드라이버에 리턴하고, 드라이버에서 엘리먼트들은 리스트로 나열된다.

## 파일로부터 데이터 읽기

텍스트 파일로부터 데이터를 읽을 때, 파일의 각 행이 RDD의 한 엘리먼트를 이룬다.

In [13]:
data_from_file.take(1)

['                   1                                          2101  M1087 432311  4M4                2014U7CN                                    I64 238 070   24 0111I64                                                                                                                                                                           01 I64                                                                                                  01  11                                 100 601']

## 람다 표현
책에서는 위의 표현이 보기 어렵다며 아래의 함수를 정의했다. 하지만 결과를 보면 그것도 딱히 보기 좋은 모양은 아니다.

In [14]:
def extractInformation(row):
    import re
    import numpy as np

    selected_indices = [
         2,4,5,6,7,9,10,11,12,13,14,15,16,17,18,
         19,21,22,23,24,25,27,28,29,30,32,33,34,
         36,37,38,39,40,41,42,43,44,45,46,47,48,
         49,50,51,52,53,54,55,56,58,60,61,62,63,
         64,65,66,67,68,69,70,71,72,73,74,75,76,
         77,78,79,81,82,83,84,85,87,89
    ]

    '''
        Input record schema
        schema: n-m (o) -- xxx
            n - position from
            m - position to
            o - number of characters
            xxx - description
        1. 1-19 (19) -- reserved positions
        2. 20 (1) -- resident status
        3. 21-60 (40) -- reserved positions
        4. 61-62 (2) -- education code (1989 revision)
        5. 63 (1) -- education code (2003 revision)
        6. 64 (1) -- education reporting flag
        7. 65-66 (2) -- month of death
        8. 67-68 (2) -- reserved positions
        9. 69 (1) -- sex
        10. 70 (1) -- age: 1-years, 2-months, 4-days, 5-hours, 6-minutes, 9-not stated
        11. 71-73 (3) -- number of units (years, months etc)
        12. 74 (1) -- age substitution flag (if the age reported in positions 70-74 is calculated using dates of birth and death)
        13. 75-76 (2) -- age recoded into 52 categories
        14. 77-78 (2) -- age recoded into 27 categories
        15. 79-80 (2) -- age recoded into 12 categories
        16. 81-82 (2) -- infant age recoded into 22 categories
        17. 83 (1) -- place of death
        18. 84 (1) -- marital status
        19. 85 (1) -- day of the week of death
        20. 86-101 (16) -- reserved positions
        21. 102-105 (4) -- current year
        22. 106 (1) -- injury at work
        23. 107 (1) -- manner of death
        24. 108 (1) -- manner of disposition
        25. 109 (1) -- autopsy
        26. 110-143 (34) -- reserved positions
        27. 144 (1) -- activity code
        28. 145 (1) -- place of injury
        29. 146-149 (4) -- ICD code
        30. 150-152 (3) -- 358 cause recode
        31. 153 (1) -- reserved position
        32. 154-156 (3) -- 113 cause recode
        33. 157-159 (3) -- 130 infant cause recode
        34. 160-161 (2) -- 39 cause recode
        35. 162 (1) -- reserved position
        36. 163-164 (2) -- number of entity-axis conditions
        37-56. 165-304 (140) -- list of up to 20 conditions
        57. 305-340 (36) -- reserved positions
        58. 341-342 (2) -- number of record axis conditions
        59. 343 (1) -- reserved position
        60-79. 344-443 (100) -- record axis conditions
        80. 444 (1) -- reserve position
        81. 445-446 (2) -- race
        82. 447 (1) -- bridged race flag
        83. 448 (1) -- race imputation flag
        84. 449 (1) -- race recode (3 categories)
        85. 450 (1) -- race recode (5 categories)
        86. 461-483 (33) -- reserved positions
        87. 484-486 (3) -- Hispanic origin
        88. 487 (1) -- reserved
        89. 488 (1) -- Hispanic origin/race recode
     '''

    record_split = re\
        .compile(
            r'([\s]{19})([0-9]{1})([\s]{40})([0-9\s]{2})([0-9\s]{1})([0-9]{1})([0-9]{2})' + 
            r'([\s]{2})([FM]{1})([0-9]{1})([0-9]{3})([0-9\s]{1})([0-9]{2})([0-9]{2})' + 
            r'([0-9]{2})([0-9\s]{2})([0-9]{1})([SMWDU]{1})([0-9]{1})([\s]{16})([0-9]{4})' +
            r'([YNU]{1})([0-9\s]{1})([BCOU]{1})([YNU]{1})([\s]{34})([0-9\s]{1})([0-9\s]{1})' +
            r'([A-Z0-9\s]{4})([0-9]{3})([\s]{1})([0-9\s]{3})([0-9\s]{3})([0-9\s]{2})([\s]{1})' + 
            r'([0-9\s]{2})([A-Z0-9\s]{7})([A-Z0-9\s]{7})([A-Z0-9\s]{7})([A-Z0-9\s]{7})' + 
            r'([A-Z0-9\s]{7})([A-Z0-9\s]{7})([A-Z0-9\s]{7})([A-Z0-9\s]{7})([A-Z0-9\s]{7})' + 
            r'([A-Z0-9\s]{7})([A-Z0-9\s]{7})([A-Z0-9\s]{7})([A-Z0-9\s]{7})([A-Z0-9\s]{7})' + 
            r'([A-Z0-9\s]{7})([A-Z0-9\s]{7})([A-Z0-9\s]{7})([A-Z0-9\s]{7})([A-Z0-9\s]{7})' + 
            r'([A-Z0-9\s]{7})([\s]{36})([A-Z0-9\s]{2})([\s]{1})([A-Z0-9\s]{5})([A-Z0-9\s]{5})' + 
            r'([A-Z0-9\s]{5})([A-Z0-9\s]{5})([A-Z0-9\s]{5})([A-Z0-9\s]{5})([A-Z0-9\s]{5})' + 
            r'([A-Z0-9\s]{5})([A-Z0-9\s]{5})([A-Z0-9\s]{5})([A-Z0-9\s]{5})([A-Z0-9\s]{5})' + 
            r'([A-Z0-9\s]{5})([A-Z0-9\s]{5})([A-Z0-9\s]{5})([A-Z0-9\s]{5})([A-Z0-9\s]{5})' + 
            r'([A-Z0-9\s]{5})([A-Z0-9\s]{5})([A-Z0-9\s]{5})([\s]{1})([0-9\s]{2})([0-9\s]{1})' + 
            r'([0-9\s]{1})([0-9\s]{1})([0-9\s]{1})([\s]{33})([0-9\s]{3})([0-9\s]{1})([0-9\s]{1})')
    try:
        rs = np.array(record_split.split(row))[selected_indices]
    except:
        rs = np.array(['-99'] * len(selected_indices))
    return rs
#     return record_split.split(row)

In [22]:
data_from_file_conv = data_from_file.map(extractInformation)
data_from_file_conv.map(lambda row: row).take(1)

[array(['1', '  ', '2', '1', '01', 'M', '1', '087', ' ', '43', '23', '11',
        '  ', '4', 'M', '4', '2014', 'U', '7', 'C', 'N', ' ', ' ', 'I64 ',
        '238', '070', '   ', '24', '01', '11I64  ', '       ', '       ',
        '       ', '       ', '       ', '       ', '       ', '       ',
        '       ', '       ', '       ', '       ', '       ', '       ',
        '       ', '       ', '       ', '       ', '       ', '01',
        'I64  ', '     ', '     ', '     ', '     ', '     ', '     ',
        '     ', '     ', '     ', '     ', '     ', '     ', '     ',
        '     ', '     ', '     ', '     ', '     ', '     ', '01', ' ',
        ' ', '1', '1', '100', '6'], dtype='<U40')]

## 전역범위 vs 지역범위

Pyspark 사용자로서 익숙해져야 할 것 중 하나는 스파크의 병렬처리다. 파이썬에 능숙하더라도 Pyspark에서 스크립트를 실행하는 것에 어려움을 느낄 수 있다.

스파크는 로컬 모드와 클러스터 모드로 동작할 수 있다. 스파크가 로컬 모드로 동작할 때는 파이썬을 실행시키는 것과 다르지 않을 수도 있다. 하지만 특별한 주의 없이 클러스터모드에서 같은 코드를 실행하면 골치아픈일이 많이 생긴다. 

클러스터 모드에서 job이 실행되면, 그 job은 마스터 노드에 보내진다. 마스터노드는 job을 위해 DAG을 생성하고 어떤 워커 노드가 특정 태스크를 실행할지 결정한다. 그리고 드라이버는 각 태스크를 마칠 준비를 한다. 워커 노드가 태스크를 수행하고 작업을 마치면 그 결과를 마스터 노드에 리턴하도록 한다. 

이 변수와 함수는 내부적으로 실행 노드의 문맥 상에서 정적이다. 즉, 각자의 실행 노드가 드라이버 노드에서 사용되는 변수와 함수를 복사해 사용한다. 태스크를 실행할 때, 실행 노드가 이 변수나 함수를 수정할 경우 다른 실행 노드들의 변수나 함수에는 영향을 주지 않는다. 이로 인해 런타임 버그나 몇몇 이상 행위를 유발할 수 있는데, 보통 이러한 오류들은 굉장히 추적하기가 어렵다. 

추가적인 예제는 http://spark.apache.org/docs/latest/programming-guide.html#local-vs-cluster-modes

## Transformation

트랜스포메이션은 데이터셋의 형태를 만든다. 이것들은 필터링, 조인, 데이터셋 내의 값들에 대한 트랜스코딩 등을 포함한다. 

가능한 모든 함수는 pyspark 문서를 참고해야 한다. 

http://spark.apache.org/docs/latest/api/python/pyspark.html#pyspark.RDD

### map()

이 함수는 RDD의 각 엘리먼트에 적용된다. 

In [23]:
data_2014 = data_from_file_conv.map(lambda row: int(row[16]))
data_2014.take(10)

[2014, 2014, 2014, 2014, 2014, 2014, 2014, 2014, 2014, -99]

column을 추가해보자

In [24]:
data_2014_2 = data_from_file_conv.map(lambda row: (row[16], int(row[16])))
data_2014_2.take(10)

[('2014', 2014),
 ('2014', 2014),
 ('2014', 2014),
 ('2014', 2014),
 ('2014', 2014),
 ('2014', 2014),
 ('2014', 2014),
 ('2014', 2014),
 ('2014', 2014),
 ('-99', -99)]

### filter()

데이터 셋으로부터 특정 조건에 맞는 엘리먼트를 선택할 수 있다.
아래 코드는 pc환경에 따라 다소 시간이 걸릴 수 있다. 개인용 pc에서는 2분 이상 걸려도 이상하지 않다

In [25]:
data_filtered = data_from_file_conv.filter(lambda row: row[5] == 'F' and row[21] == '0')
data_filtered.count()

6

### flatMap()

map() 함수와 비슷하게 동작한다. 그러나 이는 리스트가 아닌 flattened result를 리턴한다. 다음 코드의 실행 결과를 data2014_2와 비교해보자

In [27]:
data_2014_flat = data_from_file_conv.flatMap(lambda row: (row[16], int(row[16]) + 1))
data_2014_flat.take(10)

['2014', 2015, '2014', 2015, '2014', 2015, '2014', 2015, '2014', 2015]

### distinct()

이 함수는 특정 칼럼에서의 중복된 값을 제거하고, 고유한 값을 리스트로 리턴한다. 이는 데이터셋을 알아보거나 검증할 때 특히 유용하다. 

이 함수는 많은 자원을 사용하는 함수고, 데이터를 섞는 연산을 포함하기 때문에 정말 필요한 경우에만 사용해야 한다.

In [28]:
distinct_gender = data_from_file_conv.map(lambda row: row[5]).distinct().collect()
distinct_gender

['M', 'F', '-99']

### sample()

.sample() 함수는 데이터셋으로부터 임의로 추출된 샘플을 리턴한다.

- 첫번째 파라미터 : 중복 허용 여부
- 두번째 파라미터 : 리턴할 데이터셋과 전체 데이터셋 간의 크기 비율 (몇 퍼센트 비율의 샘플링을 원하는지)
- 세번째 파라미터 : 임의의 숫자를 생성하기 위한 시드

In [30]:
fraction = 0.1
data_sample = data_from_file_conv.sample(False, fraction, 777)

data_sample.take(1)

[array(['1', '  ', '6', '1', '01', 'M', '1', '074', ' ', '40', '20', '09',
        '  ', '6', 'D', '1', '2014', 'U', '7', 'C', 'N', ' ', ' ', 'I48 ',
        '228', '068', '   ', '22', '03', '11I64  ', '21I48  ', '61F03  ',
        '       ', '       ', '       ', '       ', '       ', '       ',
        '       ', '       ', '       ', '       ', '       ', '       ',
        '       ', '       ', '       ', '       ', '       ', '03',
        'I48  ', 'F03  ', 'I64  ', '     ', '     ', '     ', '     ',
        '     ', '     ', '     ', '     ', '     ', '     ', '     ',
        '     ', '     ', '     ', '     ', '     ', '     ', '01', ' ',
        ' ', '1', '1', '100', '6'], dtype='<U40')]

In [31]:
# 10% 샘플링한다
print('Original dataset: {0}, sample: {1}'.format(data_from_file_conv.count(), data_sample.count()))

Original dataset: 2631171, sample: 263053


### leftOuterJoin()

두 개의 RDD를 두 개의 데이터셋에서 찾은 값에 기반해 조인하고, 왼쪽 RDD에 오른쪽 RDD가 추가된 결과를 리턴한다.

In [34]:
rdd1 = sc.parallelize([('a', 1), ('b', 4), ('c',10)])
rdd2 = sc.parallelize([('a', 4), ('a', 1), ('b', '6'), ('d', 15)])

rdd3 = rdd1.leftOuterJoin(rdd2)
rdd3.take(5)

[('a', (1, 4)), ('a', (1, 1)), ('b', (4, '6')), ('c', (10, None))]

rdd2는 c값을 가지고 있지 않다. 그래서 None로 join되었다. d값은 rdd1에 없기 때문에 무시되었다.

만약 .join() 함수를 사용했다면, a와 b가 두 RDD의 교집합이기 때문에 a와 b만 갖게 되었을 것이다. 

In [35]:
rdd4 = rdd1.join(rdd2)
rdd4.collect()

[('a', (1, 4)), ('a', (1, 1)), ('b', (4, '6'))]

두 RDD의 교집합 엘리먼트를 리턴하는 .intersection() 함수도 유용하다.

In [36]:
rdd5 = rdd1.intersection(rdd2)
rdd5.collect()

[('a', 1)]

### repartition() 

데이터셋을 재파티션하면 데이터가 나눠지는 파티션의 개수가 바뀐다. 이 기능은 데이터를 섞어서 성능을 크게 저하시키는 요인이 되기 때문에 정말 필요할 때만 가끔 사용해야 한다.

In [37]:
rdd1 = rdd1.repartition(4)
# glom() 함수는 하나르 리스트를 생성한다.
# glom()이 생성하는 리스트는 파티션 개수만큼의 엘리먼트를 가지고 있다.
len(rdd1.glom().collect())

4

## Action

액션은 데이터셋에서 스케줄된 task를 실행한다. 

### .take()

하나의 파티션에서 가장 위에 있는 n행을 리턴하기 때문에 RDD 전체를 리턴하는 .collect()보다 더 자주 쓰인다. 큰 데이터셋일수록 특히 중요하다

In [40]:
data_first = data_from_file_conv.take(1)
data_first

[array(['1', '  ', '2', '1', '01', 'M', '1', '087', ' ', '43', '23', '11',
        '  ', '4', 'M', '4', '2014', 'U', '7', 'C', 'N', ' ', ' ', 'I64 ',
        '238', '070', '   ', '24', '01', '11I64  ', '       ', '       ',
        '       ', '       ', '       ', '       ', '       ', '       ',
        '       ', '       ', '       ', '       ', '       ', '       ',
        '       ', '       ', '       ', '       ', '       ', '01',
        'I64  ', '     ', '     ', '     ', '     ', '     ', '     ',
        '     ', '     ', '     ', '     ', '     ', '     ', '     ',
        '     ', '     ', '     ', '     ', '     ', '     ', '01', ' ',
        ' ', '1', '1', '100', '6'], dtype='<U40')]

### .reduce()

.reduce() 함수는 특정 함수를 사용해 RDD의 개수를 줄인다. RDD의 총합을 구하기 위해 이 함수를 사용할 수 있다.

reduce() 함수는 각각의 파티션에서 합계 함수를 수행하고, 마지막 집계가 수행되는 드라이버 노드에 그 합계를 리턴한다.

In [42]:
rdd1.map(lambda row: row[1]).reduce(lambda x, y: x + y)

15

In [43]:
data_reduce = sc.parallelize([1, 2, .5, .1, 5, .2], 1)
works = data_reduce.reduce(lambda x, y: x / y)
works

10.0

당연히 파티션의 수가 다르면 결과가 다르게 나온다.

In [44]:
data_reduce = sc.parallelize([1, 2, .5, .1, 5, .2], 3)
data_reduce.reduce(lambda x, y: x / y)

0.004

.reduceByKey() 함수는 .reduce() 함수와 비슷하지만 이 함수는 key값을 기반으로 reduce를 한다.

In [45]:
data_key = sc.parallelize([('a', 4),('b', 3),('c', 2),('a', 8),('d', 2),('b', 1),('d', 3)],4)
data_key.reduceByKey(lambda x, y: x + y).collect()

[('b', 4), ('c', 2), ('a', 12), ('d', 5)]

## .count()

.count() 함수는 RDD의 element 개수를 센다. 

In [46]:
data_reduce.count()

6

.count() 함수는 다음 함수와 같은 결과를 출력하나, .count() 함수는 전체 데이터셋을 드라이버로 옮기지는 않는다.

In [47]:
len(data_reduce.collect()) # WRONG -- DON'T DO THIS!

6

데이터셋이 key-value 형태로 있으면, 고유한 키의 수를 구하기 위해 .countByKey() 함수를 사용할 수도 있다. 다음 코드를 실행해보자

In [58]:
data_key.countByKey().items()

dict_items([('a', 2), ('b', 2), ('c', 1), ('d', 2)])

## .saveAsTextFile()

RDD를 텍스트 파일로 저장한다.각 파티션을 분리된 파일에 저장한다.

현재 파일 저장이 안되고 있다.

In [59]:
data_key.saveAsTextFile('data_key.txt')

Py4JJavaError: An error occurred while calling o661.saveAsTextFile.
: org.apache.hadoop.mapred.FileAlreadyExistsException: Output directory file:/C:/Users/윤승한/Desktop/git_repository/study_spark/pyspark 배우기/02_RDD/data_key.txt already exists
	at org.apache.hadoop.mapred.FileOutputFormat.checkOutputSpecs(FileOutputFormat.java:131)
	at org.apache.spark.internal.io.HadoopMapRedWriteConfigUtil.assertConf(SparkHadoopWriter.scala:287)
	at org.apache.spark.internal.io.SparkHadoopWriter$.write(SparkHadoopWriter.scala:71)
	at org.apache.spark.rdd.PairRDDFunctions$$anonfun$saveAsHadoopDataset$1.apply$mcV$sp(PairRDDFunctions.scala:1096)
	at org.apache.spark.rdd.PairRDDFunctions$$anonfun$saveAsHadoopDataset$1.apply(PairRDDFunctions.scala:1094)
	at org.apache.spark.rdd.PairRDDFunctions$$anonfun$saveAsHadoopDataset$1.apply(PairRDDFunctions.scala:1094)
	at org.apache.spark.rdd.RDDOperationScope$.withScope(RDDOperationScope.scala:151)
	at org.apache.spark.rdd.RDDOperationScope$.withScope(RDDOpe

In [54]:
def parseInput(row):
    import re
    
    pattern = re.compile(r'\(\'([a-z])\', ([0-9])\)')
    row_split = pattern.split(row)
    
    return (row_split[1], int(row_split[2]))
    
data_key_reread = sc \
    .textFile('/Users/drabast/Documents/PySpark_Data/data_key.txt') \
    .map(parseInput)
data_key_reread.collect()

Py4JJavaError: An error occurred while calling z:org.apache.spark.api.python.PythonRDD.collectAndServe.
: org.apache.hadoop.mapred.InvalidInputException: Input path does not exist: file:/Users/drabast/Documents/PySpark_Data/data_key.txt
	at org.apache.hadoop.mapred.FileInputFormat.singleThreadedListStatus(FileInputFormat.java:287)
	at org.apache.hadoop.mapred.FileInputFormat.listStatus(FileInputFormat.java:229)
	at org.apache.hadoop.mapred.FileInputFormat.getSplits(FileInputFormat.java:315)
	at org.apache.spark.rdd.HadoopRDD.getPartitions(HadoopRDD.scala:204)
	at org.apache.spark.rdd.RDD$$anonfun$partitions$2.apply(RDD.scala:253)
	at org.apache.spark.rdd.RDD$$anonfun$partitions$2.apply(RDD.scala:251)
	at scala.Option.getOrElse(Option.scala:121)
	at org.apache.spark.rdd.RDD.partitions(RDD.scala:251)
	at org.apache.spark.rdd.MapPartitionsRDD.getPartitions(MapPartitionsRDD.scala:49)
	at org.apache.spark.rdd.RDD$$anonfun$partitions$2.apply(RDD.scala:253)
	at org.apache.spark.rdd

### .foreach()

RDD의 각 엘리먼트에 같은 함수를 반복적으로 적용하는 함수다. 다음 코드를 실행하면 data_key RDD에 저장된 모든 데이터가 출력된다. 단 , CLI에 출력된다.

In [57]:
def f(x): 
    print(x)

data_key.foreach(f)